Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# How to Setup a Schedule for a Pipeline
In this notebook, we will see how we can run an already published pipeline on a schedule.

## Prerequisites and AML Basics
Make sure you go through the [00.aml-pipelines-configuration](./00.aml-pipelines-configuration.ipynb) Notebook first if you haven't.

### Initialization Steps

In [ ]:
import azureml.core
from azureml.core import Workspace, Run, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute import DataFactoryCompute
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData, StepSequence
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.steps import DataTransferStep
from azureml.pipeline.core import PublishedPipeline
from azureml.pipeline.core.graph import PipelineParameter

print("Pipeline SDK-specific imports completed")

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore (Azure file storage)
def_file_store = ws.get_default_datastore() 
print("Default datastore's name: {}".format(def_file_store.name))

def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))

# project folder
project_folder = './scripts'

### Compute Targets
#### Retrieve an already attached BatchAI cluster

In [ ]:
compute_name = "hsBatch"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size, # NC6 is GPU-enabled
                                                                vm_priority = 'lowpriority', # optional
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current BatchAI cluster status, use the 'status' property    
    print(compute_target.status.serialize())

## Build and Publish Pipeline
Build a simple pipeline, publish it and add a schedule to run it.

### Define steps

In [ ]:
trainStep = PythonScriptStep(
    name="Best_Traing_Step_Ever",
    script_name="train.py", 
    compute_target=compute_target, 
    source_directory=project_folder
)

print("trainStep created")

extractStep = PythonScriptStep(
    name="Extract_Step",
    script_name="extract.py",
    compute_target=compute_target, 
    source_directory=project_folder)

print("extractStep created")

pipeline_param = PipelineParameter(name="pipeline_arg", default_value=10)

compareStep = PythonScriptStep(
    name="Compare_Step",
    arguments=["--pipeline_param", pipeline_param],   
    script_name="compare.py",
    compute_target=compute_target, 
    source_directory=project_folder)

print("compareStep created")

steps = [trainStep, extractStep, compareStep]

### Build the pipeline

In [ ]:
pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built")

pipeline1.validate()

### Publish the pipeline

In [ ]:
from datetime import datetime
timenow = datetime.now().strftime('%m-%d-%Y-%H-%M')

pipeline_name = timenow + "-Pipeline"
print(pipeline_name)

published_pipeline1 = pipeline1.publish(
    name=pipeline_name, 
    description=pipeline_name)
print(published_pipeline1.id)

### Create a schedule

In [ ]:
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
recurrence = ScheduleRecurrence(frequency="Day", interval=6, hours=[10, 12])

schedule = Schedule.create(workspace=ws, name="My_Schedule",
                           pipeline_id=published_pipeline1.id, experiment_name='Schedule_Run',
                           recurrence=recurrence, description="Schedule Run",
                           pipeline_parameters={'pipeline_arg': '20'})

print('Created schedule with id:', schedule.id)

### Get the schedule

In [ ]:
fetched_schedule = Schedule.get_schedule(ws, schedule.id)
print('Got schedule with id:', fetched_schedule.id)

### Disable the schedule

In [ ]:
fetched_schedule.disable()
fetched_schedule = Schedule.get_schedule(ws, schedule.id)
print('Disabled schedule, new status:', fetched_schedule.status)

### Reactivate the schedule

In [ ]:
fetched_schedule.activate()
fetched_schedule = Schedule.get_schedule(ws, schedule.id)
print('Activated schedule, new status:', fetched_schedule.status)

### Change reccurence of the schedule

In [ ]:
recurrence = ScheduleRecurrence(frequency="Hour", interval=2, hours=[], minutes=[20, 40])
fetched_schedule.update(name="My_Updated_Schedule", 
                        description="Updated_Schedule_Run", 
                        pipeline_parameters={'pipeline_arg': '30'},
                        status='Disabled', recurrence=recurrence)
fetched_schedule = Schedule.get_schedule(ws, fetched_schedule.id)

print('Updated schedule.',
      'New name:', fetched_schedule.name, 
      'New frequency:', fetched_schedule.recurrence.frequency,
      'New status:', fetched_schedule.status)